In [317]:
import os
dirpath = os.getcwd()
directory_path = "/Users/admin/Documents/Skimming/tree_of_life"
report_directory = "report_kraken_viral_dist_unmasked_0.0_corrected_k35"
os.chdir(directory_path)
print (os.getcwd())

/Users/admin/Documents/Skimming/tree_of_life


In [318]:
from pandas import DataFrame
import pandas as pd
import numpy as np
df = pd.read_csv('Keep_exclude_corrected.csv', header = 0)

In [319]:
df.tail(n=5)
#total_rows = df.count

,Keep,Exclude,Distance
115,NaN,10x_Oryza_sativa,10.145871
116,NaN,10x_Prunus_persica,10.145871
117,NaN,15x_Arabidopsis_lyrata,10.145871
118,NaN,20x_Arabidopsis_thaliana,10.145871
119,NaN,250x_Bathycoccus_prasinos,10.145871


In [320]:
df_metadata = pd.read_csv("metadata.tsv", sep='\t', header = 0)

In [321]:
df_metadata.head(n=2)
#total_rows = df_metadata.count
#print (total_rows)

,#genome,asm_name,assembly_accession,bioproject,biosample,wgs_master,seq_rel_date,submitter,ftp_path,img_id,...,coding_density,completeness,contamination,strain_heterogeneity,markers,5s_rrna,16s_rrna,23s_rrna,trnas,draft_quality
0,G000005825,ASM582v2,GCF_000005825.2,PRJNA224116,SAMN02603086,NaN,2010/12/15,"Center for Genomic Sciences, Allegheny-Singer ...",ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,646311908,...,85.144124,98.68,1.32,0.0,377,yes,yes,yes,20,high
1,G000006175,ASM617v2,GCF_000006175.1,PRJNA224116,SAMN00000040,NaN,2010/06/03,US DOE Joint Genome Institute (JGI-PGF),ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,646564549,...,80.167033,99.05,0.00,0.0,165,no,yes,yes,19,medium


In [322]:
#change keep column name from "keep" to #genome
df.columns
df = df.rename({'Keep': '#genome'}, axis=1) 
df.head(n=5)

,#genome,Exclude,Distance
0,G001587575,G001587655,0.011628
1,G001418715,G900109425,0.010484
2,G001437625,G001438895,0.009260
3,G900079115,G000007005,0.011244
4,G000022425,G000245135,0.009330


In [323]:
#read metadata file
df = df.merge(df_metadata[["#genome","taxid","species_taxid", "superkingdom", "species", "completeness", "contamination"]],how='left')
#df.fillna(0)

In [324]:
#update superkingdom information for plants that are not in metadata
# Viridiplantae is a kingdom
#lineage is cellular organisms (R1); Eukaryota (D); Viridiplantae (K)
df[["superkingdom"]] = df[["superkingdom"]].fillna(value = "Viridiplantae")
df[["taxid"]] = df[["taxid"]].fillna(value = 33090)

In [325]:
df.tail(n=5)

,#genome,Exclude,Distance,taxid,species_taxid,superkingdom,species,completeness,contamination
115,NaN,10x_Oryza_sativa,10.145871,33090.0,NaN,Viridiplantae,NaN,NaN,NaN
116,NaN,10x_Prunus_persica,10.145871,33090.0,NaN,Viridiplantae,NaN,NaN,NaN
117,NaN,15x_Arabidopsis_lyrata,10.145871,33090.0,NaN,Viridiplantae,NaN,NaN,NaN
118,NaN,20x_Arabidopsis_thaliana,10.145871,33090.0,NaN,Viridiplantae,NaN,NaN,NaN
119,NaN,250x_Bathycoccus_prasinos,10.145871,33090.0,NaN,Viridiplantae,NaN,NaN,NaN


In [326]:
df.insert(3, "prct_difference", 0)
df.insert(4, "prct_difference_bin", 0)

In [327]:
#Jukes_Cantor distance to difference conversion
import math
def calculate_difference(distance):
    prct_dif = 3/4*(1-(math.exp(-4/3*distance)))*100
    return prct_dif
df['prct_difference'] = df.Distance.apply(calculate_difference)

In [328]:
df.head(n=5)

,#genome,Exclude,Distance,prct_difference,prct_difference_bin,taxid,species_taxid,superkingdom,species,completeness,contamination
0,G001587575,G001587655,0.011628,1.153830,0,1706438.0,1706438.0,Archaea,Arc I group archaeon BMIXfssc0709_Meth_Bin006,90.98,2.72
1,G001418715,G900109425,0.010484,1.041081,0,277988.0,277988.0,Archaea,Thermococcus thioreducens,98.40,0.00
2,G001437625,G001438895,0.009260,0.920275,0,1655561.0,1655561.0,Archaea,Nitrosopumilus sp. BACL13 MAG-121220-bin23,99.51,0.97
3,G900079115,G000007005,0.011244,1.115971,0,2287.0,2287.0,Archaea,Sulfolobus solfataricus,100.00,0.00
4,G000022425,G000245135,0.009330,0.927176,0,427318.0,43080.0,Archaea,Sulfolobus islandicus,99.40,0.00


In [329]:
#function to add column with bins
def bin_difference(distance):
    if distance <=0.007550449:
        return 0
    elif distance >=0.007550449 and distance<=0.012584082:
        return 1
    elif distance >=0.015203628 and distance<=0.02533938:
        return 2
    elif distance >=0.030829633 and distance<=0.051382722:
        return 4
    elif distance >=0.046902155 and distance<=0.078170258:
        return 6
    elif distance >=0.063447465 and distance<=0.105745776:
        return 8
    elif distance >=0.080494225 and distance<=0.134157041:
        return 10
    elif distance >=0.134972252 and distance<=0.224953753:
        return 16
    elif distance >=0.174462147 and distance<=0.290770245:
        return 20
    elif distance >=0.428703779 and distance<=0.714506299:
        return 40
    elif distance >=0.905308826 and distance<=1.508848043:
        return 60
    else:
        return 75
    
df['prct_difference_bin'] = df.Distance.apply(bin_difference)


In [330]:
#add calculated mash distances to pairwise tree distances
mash_file = "/Users/admin/Documents/Skimming/tree_of_life/mash_dist_output.txt"
df_mash = pd.read_csv(mash_file, sep="\t", header=None, names=["#genome2", "Exclude2", "mash_distance", "p_value", "matching_hashes"])
df = pd.concat([df.reset_index(drop=True),df_mash.reset_index(drop=True)], axis=1)
df = df.drop(columns=["#genome2", "Exclude2", "p_value", "matching_hashes"])
#calculated mash doesn't contain values for plants so filling them in as mash distance 1
df[["mash_distance"]] = df[["mash_distance"]].fillna(value = 1)

In [331]:
df.head(n=2)

,#genome,Exclude,Distance,prct_difference,prct_difference_bin,taxid,species_taxid,superkingdom,species,completeness,contamination,mash_distance
0,G001587575,G001587655,0.011628,1.153830,1,1706438.0,1706438.0,Archaea,Arc I group archaeon BMIXfssc0709_Meth_Bin006,90.98,2.72,0.001653
1,G001418715,G900109425,0.010484,1.041081,1,277988.0,277988.0,Archaea,Thermococcus thioreducens,98.40,0.00,0.000216


In [332]:
#look up 10K mash distance between # genomes and exclude and populate outer column
file1 = "/Users/admin/Documents/Skimming/tree_of_life/batch2.dm"
df_distance_10Kmatrix = pd.read_csv(file1, sep='\t', header = 0)
df["mash_10K_distance"] = 0

#get pairwise mash distances for query and keep nodes from 
df_distance_10Kmatrix = df_distance_10Kmatrix.rename({'Unnamed: 0': 'node_clmn'}, axis=1) 
dict_distance_10Kmatrix = {}
for i in range(0, len(df)):
    keep_val = str(df.iloc[i,0])
    query_val = str(df.iloc[i,1])
    try:
        df.loc[i,"mash_10K_distance"] = float(df_distance_10Kmatrix.loc[df_distance_10Kmatrix.node_clmn == keep_val, query_val])
    except:
        df.loc[i,"mash_10K_distance"] = 1

In [333]:
#populate mean, median for query nodes from 10K matrix
df_stats = pd.read_csv('10K_dist_stats.csv', header = 0)
df_stats = df_stats.rename({"query_seq": "Exclude"}, axis=1)
df = df.merge(df_stats[["Exclude", "min_dist","neighborhood_mean","neighborhood_median"]],how='left')
#df = pd.merge(left=df,right=df_stats, left_on="Exclude", right_on="query_seq", how="left")

In [334]:
df.head(n=2)

,#genome,Exclude,Distance,prct_difference,prct_difference_bin,taxid,species_taxid,superkingdom,species,completeness,contamination,mash_distance,mash_10K_distance,min_dist,neighborhood_mean,neighborhood_median
0,G001587575,G001587655,0.011628,1.153830,1,1706438.0,1706438.0,Archaea,Arc I group archaeon BMIXfssc0709_Meth_Bin006,90.98,2.72,0.001653,0.001653,0.001653,0.198533,0.230110
1,G001418715,G900109425,0.010484,1.041081,1,277988.0,277988.0,Archaea,Thermococcus thioreducens,98.40,0.00,0.000216,0.000216,0.000216,0.161583,0.182467


In [335]:
df.insert(5, "C_sequences", 0)
df.insert(6, "U_sequences", 0)
df.insert(7, "C_sequences_prct", 0.0)
df.insert(8, "U_sequences_prct", 0.0)

df.insert(9, "num_of_neighbor_seqs", 0)
df.insert(10, "prct_of_neighbor_seqs", 0.0)
df.insert(11, "species_detected", "")

#add column to track human sequences since db base is human (taxid: 9606)
df.insert(12, "num_of_human_seqs", 0)
df.insert(13, "prct_human", 0.0)

df.insert(14, "num_of_domain_seqs", 0)
df.insert(15, "prct_domain", 0.0)
df.insert(16, "num_of_kingdom_seqs", 0)
df.insert(17, "prct_kingdom", 0.0)
df.insert(18, "num_of_phylum_seqs", 0)
df.insert(19, "prct_phylum", 0.0)
df.insert(20, "num_of_class_seqs", 0)
df.insert(21, "prct_class", 0.0)
df.insert(22, "num_of_order_seqs", 0)
df.insert(23, "prct_order", 0.0)
df.insert(24, "num_of_family_seqs", 0)
df.insert(25, "prct_family", 0.0)
df.insert(26, "num_of_genus_seqs", 0)
df.insert(27, "prct_genus", 0.0)
df.insert(28, "num_of_species_seqs", 0)
df.insert(29, "prct_species", 0.0)


df["TP"] = 0
df["TN"] = 0
df["FP"] = 0
df["FN"] = 0

#total number of sequences to reach 1G
total_sequences = 6666667

In [336]:
#change working directory
def change_wd (folder_path):
    if os.path.exists(folder_path) :
        # Change the current working Directory    
        os.chdir(folder_path)
    else:
        print("Can't change the Current Working Directory")    

In [337]:
#go to specified directory and get values of k, l, s, confidence score, masking
import numbers
directory_in_str = directory_path + "/" + report_directory
change_wd(directory_in_str)

#get k values
k_value = int(report_directory.split("k")[-1])

#get l value
if k_value == 35:
    l_value = 31
else:
    l_value = k_value
    
#get s value
if l_value == 31:
    s_value = 7
elif l_value == 26:
    s_value = 6
elif l_value == 20:
    s_value = 5
elif l_value == 16:
    s_value = 4

#get confidence_level
folder_name_split = (report_directory.split("_"))
confidence_level = 0
for x in folder_name_split:
    try:
        x = float(x)
        confidence_level = x
        break
    except:
        continue
        
# get masked vs unmasked info about DB
for x in folder_name_split:
    if x == "masked":
        masking = "masked"    
    else:
        masking = "unmasked"
        
#get file list
files = []
for file in sorted(os.listdir(directory_in_str)):
    files.append(file)

In [338]:
#get file_name based on match  genome and report names:
def get_filename (genome, file_list):
    for fi in file_list:
        if genome in fi:
            return fi           

In [339]:
#A rank code, indicating (U)nclassified, (R)oot, (D)omain, (K)ingdom,
# (P)hylum, (C)lass, (O)rder, (F)amily, (G)enus, or (S)pecies.
def get_rank_stats(df, rank): 
    row = df.loc[df["rank_code"] == rank]
    row_index = df.index[df["rank_code"] == rank].tolist()
    return row, row_index
def get_taxid_stats(df, taxid): 
    row = df.loc[df["NCBI_taxonomic_ID"] == taxid]
    row_index = df.index[df["NCBI_taxonomic_ID"] == taxid].tolist()
    return row, row_index

In [340]:
#construct tax ID list of interest 
# for standard DB
bacteria_D_taxID = 2
archaea_D_taxID = 2157
viruses_D_taxID = 10239
eukaryota_D_taxID = 2759

viridiplantae_K_taxID = 33090
human_S_taxID = 9606

In [341]:
#function to calculate number of sequences classified to bacteria and archaea for plants
def get_FP_for_plants(df):
    bacteria_count = 0
    bac_row = df.loc[df["NCBI_taxonomic_ID"] == bacteria_D_taxID]
    bac_row_index = df.index[df["NCBI_taxonomic_ID"] == bacteria_D_taxID].tolist()
    if len(bac_row_index)>0:
        bacteria_count = bac_row.iat[0,1]
    arch_count = 0
    arch_row = df.loc[df["NCBI_taxonomic_ID"] == archaea_D_taxID]
    arch_row_index = df.index[df["NCBI_taxonomic_ID"] == archaea_D_taxID].tolist()
    if len(arch_row_index)>0:
        arch_count = arch_row.iat[0,1]  
    total = bacteria_count + arch_count
    return total

In [342]:
#function to calculate how many sequences classified at (D)omain, (K)ingdom,
#(P)hylum, (C)lass, (O)rder, (F)amily, (G)enus, or (S)pecies.
def get_level_counts_for_report(df, start_index):
    counts = {"K":0, "P":0, "C":0, "O":0, "F":0, "G":0, "S":0}
    prcts = {"K":0, "P":0, "C":0, "O":0, "F":0, "G":0, "S":0}
    #find the the row where next Domain starts
    df = df[start_index:]
    D_index = df.index[df["rank_code"] == "D"].tolist()
    
    #slice data frame to get values for a single Domain
    if len(D_index) >1:
        df_slice = df[1:D_index[1]]
    else:
        df_slice = df[1:] 
    
    if len(df_slice)>0:       
        counts["K"]= df_slice.loc[df_slice["rank_code"] == "K", "num_of_frgmts_covered_by_clade"].sum()
        prcts["K"]= df_slice.loc[df_slice["rank_code"] == "K", "pct_of_frgmts_covered_by_clade"].sum()
        counts["P"]= df_slice.loc[df_slice["rank_code"] == "P", "num_of_frgmts_covered_by_clade"].sum()
        prcts["P"]= df_slice.loc[df_slice["rank_code"] == "P", "pct_of_frgmts_covered_by_clade"].sum()      
        counts["C"]= df_slice.loc[df_slice["rank_code"] == "C", "num_of_frgmts_covered_by_clade"].sum()
        prcts["C"]= df_slice.loc[df_slice["rank_code"] == "C", "pct_of_frgmts_covered_by_clade"].sum()        
        counts["O"]= df_slice.loc[df_slice["rank_code"] == "O", "num_of_frgmts_covered_by_clade"].sum()
        prcts["O"]= df_slice.loc[df_slice["rank_code"] == "O", "pct_of_frgmts_covered_by_clade"].sum()    
        counts["F"]= df_slice.loc[df_slice["rank_code"] == "F", "num_of_frgmts_covered_by_clade"].sum()
        prcts["F"]= df_slice.loc[df_slice["rank_code"] == "F", "pct_of_frgmts_covered_by_clade"].sum()          
        counts["G"]= df_slice.loc[df_slice["rank_code"] == "G", "num_of_frgmts_covered_by_clade"].sum()
        prcts["G"]= df_slice.loc[df_slice["rank_code"] == "G", "pct_of_frgmts_covered_by_clade"].sum()          
        counts["S"]= df_slice.loc[df_slice["rank_code"] == "S", "num_of_frgmts_covered_by_clade"].sum()
        prcts["S"]= df_slice.loc[df_slice["rank_code"] == "S", "pct_of_frgmts_covered_by_clade"].sum()                     
    
    return counts, prcts

In [343]:
#iterate over rows in data frame
for row in df.itertuples():
    #open corresponding reports, parse and populate data
    genome = row.Exclude
    filename = get_filename (genome, files)
    F = open(directory_in_str + "/" + filename)
    report = pd.read_csv(F, sep="\t", header=None, names=["pct_of_frgmts_covered_by_clade", "num_of_frgmts_covered_by_clade", "num_of_frgmts_assigned_to_taxon", "rank_code", "NCBI_taxonomic_ID", "scientific_name"])
    F.close()
    
    #populate data frame with total C and U information
    U_row, U_row_index = get_rank_stats(report, "U")
    R_row, R_row_index = get_rank_stats(report, "R")
    if len(U_row_index)>0:
        df.loc[row.Index, "U_sequences"] = U_row.iat[0, 1]
        df.loc[row.Index, "U_sequences_prct"] = U_row.iat[0, 0]
    if len(R_row_index)>0:
        df.loc[row.Index, "C_sequences"] = R_row.iat[0, 1]
        df.loc[row.Index, "C_sequences_prct"] = R_row.iat[0, 0]
        
    #populate data frame with sequences classified to the closest neighbor
    target_taxid = row.taxid
    neighbor_row, neighbor_row_index = get_taxid_stats(report, target_taxid)
    if len(neighbor_row_index)>0:
        df.loc[row.Index, "num_of_neighbor_seqs"] = neighbor_row.iat[0, 1]
        df.loc[row.Index, "prct_of_neighbor_seqs"] = neighbor_row.iat[0, 0]
        df.loc[row.Index, "species_detected"] = neighbor_row.iat[0, 5]
        
    #populate data frame with sequences classified to human
    #human is the base for Kraken DB and needs to tbe substituted with vorus for viral DB
    human_row, human_row_index = get_taxid_stats(report, human_S_taxID)
    if len(human_row_index)>0:
        df.loc[row.Index, "num_of_human_seqs"] = human_row.iat[0, 1]
        df.loc[row.Index, "prct_human"] = human_row.iat[0, 0]
    
    #populate data frame with sequences classified to expected Domain : Bacteria or Archaea
    #also equal to TP for confusion matrix
    #count as bacteria+archaea at D level
    
    expected_domain = row.superkingdom
    count_D_seqs = 0
    count_prct_D_seqs = 0
    
        
    if expected_domain == "Bacteria" or expected_domain == "Archaea":
        bac_D_row, bac_D_row_index = get_taxid_stats(report, bacteria_D_taxID)  
        if len(bac_D_row_index)>0:
            count_D_seqs += bac_D_row.iat[0, 1]
            count_prct_D_seqs += bac_D_row.iat[0, 0]
            #get counts for other levels within given domain
            bac_counts, bac_prcts = get_level_counts_for_report(report, bac_D_row_index[0])

        arch_D_row, arch_D_row_index = get_taxid_stats(report, archaea_D_taxID)
        if len(arch_D_row_index)>0:
            count_D_seqs += arch_D_row.iat[0, 1]
            count_prct_D_seqs += arch_D_row.iat[0, 0]
            #get counts for other levels within given domain
            arch_counts, arch_prcts = get_level_counts_for_report(report, arch_D_row_index[0])
            
        df.loc[row.Index, "num_of_domain_seqs"] = count_D_seqs
        df.loc[row.Index, "prct_domain"] = count_prct_D_seqs           
        df.loc[row.Index, "num_of_kingdom_seqs"] = bac_counts["K"] + arch_counts["K"]
        df.loc[row.Index, "prct_kingdom"] = bac_prcts["K"] + arch_prcts["K"]   
        df.loc[row.Index, "num_of_phylum_seqs"] = bac_counts["P"] + arch_counts["P"]
        df.loc[row.Index, "prct_phylum"] = bac_prcts["P"] + arch_prcts["P"] 
        df.loc[row.Index, "num_of_class_seqs"] = bac_counts["C"] + arch_counts["C"]
        df.loc[row.Index, "prct_class"] = bac_prcts["C"] + arch_prcts["C"] 
        df.loc[row.Index, "num_of_order_seqs"] = bac_counts["O"] + arch_counts["O"]
        df.loc[row.Index, "prct_order"] = bac_prcts["O"] + arch_prcts["O"] 
        df.loc[row.Index, "num_of_family_seqs"] = bac_counts["F"] + arch_counts["F"]
        df.loc[row.Index, "prct_family"] = bac_prcts["F"] + arch_prcts["F"] 
        df.loc[row.Index, "num_of_genus_seqs"] = bac_counts["G"] + arch_counts["G"]
        df.loc[row.Index, "prct_genus"] = bac_prcts["G"] + arch_prcts["G"] 
        df.loc[row.Index, "num_of_species_seqs"] = bac_counts["S"] + arch_counts["S"]
        df.loc[row.Index, "prct_species"] = bac_prcts["S"] + arch_prcts["S"] 
        
           
    elif expected_domain == "Viruses":
        target_taxid = viruses_D_taxID
        D_row, D_row_index = get_taxid_stats(report, target_taxid)
        if len(D_row_index)>0:
            df.loc[row.Index, "num_of_domain_seqs"] = D_row.iat[0, 1]
            df.loc[row.Index, "prct_domain"] = D_row.iat[0, 0]    
                
    elif expected_domain == "Viridiplantae":
        target_taxid = viridiplantae_K_taxID
        D_row, D_row_index = get_taxid_stats(report, target_taxid)
        if len(D_row_index)>0:
            df.loc[row.Index, "num_of_domain_seqs"] = D_row.iat[0, 1]
            df.loc[row.Index, "prct_domain"] = D_row.iat[0, 0]     
    
    
    if row.Index <= 109:
        #calculate TP as Bacteria classified to bacteria or archaea
        df.loc[row.Index, "TP"] = df.loc[row.Index, "num_of_domain_seqs"]
        #calculate FN= total - FP
        df.loc[row.Index, "FN"] = total_sequences - df.loc[row.Index, "TP"]
    else:
        #calculate FP as Plants classified to bacteria or archaea
        df.loc[row.Index, "FP"] = get_FP_for_plants(report)
        #calculate FP as Plants classified to bacteria or archaea
        df.loc[row.Index, "TN"] = total_sequences - df.loc[row.Index, "FP"]

In [344]:
df

,#genome,Exclude,Distance,prct_difference,prct_difference_bin,C_sequences,U_sequences,C_sequences_prct,U_sequences_prct,num_of_neighbor_seqs,...,contamination,mash_distance,mash_10K_distance,min_dist,neighborhood_mean,neighborhood_median,TP,TN,FP,FN
0,G001587575,G001587655,0.011628,1.153830,1,6438414,228250,96.58,3.42,105901,...,2.72,0.001653,0.001653,0.001653,0.198533,0.230110,6437628,0,0,229039
1,G001418715,G900109425,0.010484,1.041081,1,6650752,15915,99.76,0.24,6608553,...,0.00,0.000216,0.000216,0.000216,0.161583,0.182467,6650195,0,0,16472
2,G001437625,G001438895,0.009260,0.920275,1,6633257,33409,99.50,0.50,6548538,...,0.97,0.017184,0.017184,0.017184,0.164008,0.187000,6632791,0,0,33876
3,G900079115,G000007005,0.011244,1.115971,1,6635833,20966,99.54,0.31,5822393,...,0.00,0.003344,0.003344,0.003344,0.034572,0.008099,6632769,0,0,33898
4,G000022425,G000245135,0.009330,0.927176,1,6662936,2777,99.94,0.04,17417,...,0.00,0.000781,0.000781,0.000781,0.002879,0.002720,6662823,0,0,3844
5,G001560285,G001481595,0.011083,1.100121,1,6666666,1,100.00,0.00,6666436,...,0.00,0.000144,0.000144,0.000072,0.000088,0.000096,6666666,0,0,1
6,G001560515,G001560565,0.010205,1.013619,1,6666667,0,100.00,0.00,6666470,...,0.00,0.000557,0.000557,0.000072,0.000091,0.000096,6666667,0,0,0
7,G001560115,G001560165,0.010684,1.060791,1,6666666,1,100.00,0.00,6666446,...,0.00,0.000144,0.000144,0.000096,0.000127,0.000119,6666666,0,0,1
8,G000371805,G000363885,0.011585,1.149598,1,6666667,0,100.00,0.00,6658875,...,0.24,0.001109,0.001109,0.001109,0.244806,0.279502,6666667,0,0,0
9,G000190135,G000190155,0.012389,1.228762,1,6666460,207,100.00,0.00,135940,...,0.00,0.001418,0.001418,0.001418,0.133596,0.137168,6666310,0,0,357


In [345]:
#add k, l, s, confidence level and masking info to the file information to the file
df['k'] = k_value
df['l'] = l_value
df['s'] = s_value
df['confidence_level'] = confidence_level
df['masking'] = masking
df["classifier"] = "k=" + str(k_value) + " " + "conf=" + str(confidence_level)

In [346]:
df

,#genome,Exclude,Distance,prct_difference,prct_difference_bin,C_sequences,U_sequences,C_sequences_prct,U_sequences_prct,num_of_neighbor_seqs,...,TP,TN,FP,FN,k,l,s,confidence_level,masking,classifier
0,G001587575,G001587655,0.011628,1.153830,1,6438414,228250,96.58,3.42,105901,...,6437628,0,0,229039,35,31,7,0.0,unmasked,k=35 conf=0.0
1,G001418715,G900109425,0.010484,1.041081,1,6650752,15915,99.76,0.24,6608553,...,6650195,0,0,16472,35,31,7,0.0,unmasked,k=35 conf=0.0
2,G001437625,G001438895,0.009260,0.920275,1,6633257,33409,99.50,0.50,6548538,...,6632791,0,0,33876,35,31,7,0.0,unmasked,k=35 conf=0.0
3,G900079115,G000007005,0.011244,1.115971,1,6635833,20966,99.54,0.31,5822393,...,6632769,0,0,33898,35,31,7,0.0,unmasked,k=35 conf=0.0
4,G000022425,G000245135,0.009330,0.927176,1,6662936,2777,99.94,0.04,17417,...,6662823,0,0,3844,35,31,7,0.0,unmasked,k=35 conf=0.0
5,G001560285,G001481595,0.011083,1.100121,1,6666666,1,100.00,0.00,6666436,...,6666666,0,0,1,35,31,7,0.0,unmasked,k=35 conf=0.0
6,G001560515,G001560565,0.010205,1.013619,1,6666667,0,100.00,0.00,6666470,...,6666667,0,0,0,35,31,7,0.0,unmasked,k=35 conf=0.0
7,G001560115,G001560165,0.010684,1.060791,1,6666666,1,100.00,0.00,6666446,...,6666666,0,0,1,35,31,7,0.0,unmasked,k=35 conf=0.0
8,G000371805,G000363885,0.011585,1.149598,1,6666667,0,100.00,0.00,6658875,...,6666667,0,0,0,35,31,7,0.0,unmasked,k=35 conf=0.0
9,G000190135,G000190155,0.012389,1.228762,1,6666460,207,100.00,0.00,135940,...,6666310,0,0,357,35,31,7,0.0,unmasked,k=35 conf=0.0


In [347]:
from pathlib import Path
p = Path('/Users/admin/Documents/Skimming/tree_of_life/tree_k_titration_recalc_viral/')
df.to_csv(Path(p, 'summary_' + report_directory + '.csv'), index = None, header = True)